<a   href="https://colab.research.google.com/github/N-Nieto/OHBM_SEA-SIG_Educational_Course/blob/master/03_pitfalls/03_02_data_leakage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### If you are running in Google Colab, uncomment the cell bellow to load the data.
### If you are running locally, ignore the above cell.

In [ ]:
# from pathlib import Path
# from urllib.request import urlretrieve
# # Clean files
# import pandas as pd
# import numpy as np

# # Download necessary data files
# Path("data").mkdir(exist_ok=True)

# # 01_basic_ML.ipynb needs this files
# urlretrieve('https://zenodo.org/records/17056022/files/cleaned_VBM_GM_Schaefer100x17_mean_aggregation.csv?download=1', './data/cleaned_VBM_GM_Schaefer100x17_mean_aggregation.csv')
# urlretrieve('https://zenodo.org/records/17056022/files/cleaned_IXI_behavioural.csv?download=1', './data/cleaned_IXI_behavioural.csv')

# # 02_XAI.ipynb needs also this files
# urlretrieve('https://zenodo.org/records/17056022/files/cleaned_VBM_GM_TianxS1x3TxMNI6thgeneration_mean_aggregation.csv?download=1', './data/cleaned_VBM_GM_TianxS1x3TxMNI6thgeneration_mean_aggregation.csv')

# # Load data
# df_behav = pd.read_csv("data/cleaned_IXI_behavioural.csv", index_col=0)

# # Some height values are not sensible, we filter them out
# height = df_behav["HEIGHT"].values
# df_behav = df_behav[np.logical_and(height > 120, height < 200)]

# # Remove NaNs and duplicates
# df_behav.dropna(inplace=True)
# df_behav.drop_duplicates(keep='first', inplace=True)
# df_behav.to_csv('data/cleaned_IXI_behavioural.csv')

# # Remove NaNs
# df_cortical_100 = pd.read_csv("data/cleaned_VBM_GM_Schaefer100x17_mean_aggregation.csv", index_col=0)
# df_cortical_100.dropna(inplace=True)
# df_cortical_100.to_csv('data/cleaned_VBM_GM_Schaefer100x17_mean_aggregation.csv')

# # Remove NaNs
# df_subcortical = pd.read_csv("data/cleaned_VBM_GM_TianxS1x3TxMNI6thgeneration_mean_aggregation.csv", index_col=0)
# df_subcortical.dropna(inplace=True)
# df_subcortical.to_csv('data/cleaned_VBM_GM_TianxS1x3TxMNI6thgeneration_mean_aggregation.csv')

# data_path = Path("data/")


# Leakage exploration

In [ ]:
# Import modules
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from pathlib import Path

if 'data_path' not in locals():
    data_path = Path("../data/")

In [ ]:
# Prepare the data
# Features: Cortical + Subcortical
features = ["cortical", "subcortical"]

# Target: Sex
target = ["SEX_ID (1=m, 2=f)"]
# Confounding variables: No for this example
confounding = []

df_data = pd.read_csv(data_path / "cleaned_IXI_behavioural.csv", index_col=0)
columns_features = []
for feature in features:
    if feature == "cortical":
        df_feature = pd.read_csv(
            data_path / "cleaned_VBM_GM_Schaefer100x17_mean_aggregation.csv",
            index_col=0,
        )
    elif feature == "subcortical":
        df_feature = pd.read_csv(
            data_path
            / "cleaned_VBM_GM_TianxS1x3TxMNI6thgeneration_mean_aggregation.csv",
            index_col=0,
        )
    else:
        print("feature not recognized")

    df_data = df_data.join(df_feature, how="inner")
    columns_features = columns_features + df_feature.columns.to_list()

print(f"Final data shape: {df_data.shape}")

y = df_data[target].values.ravel()
if target == ["SEX_ID (1=m, 2=f)"]:
    y = np.where(y == 2, 0, 1)  # Put the classes as 0 and 1

X = df_data.loc[:, columns_features].values  # only brain features

print("X shape")
print(X.shape)


In [ ]:
test_size = 0.3
random_state = 42
# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, shuffle=True, random_state=random_state
)
# Check size of data
print("X shape", X.shape)
print("y shape", y.shape)
print("X_train shape", X_train.shape)
print("X_test shape", X_test.shape)
print("y_train shape", y_train.shape)
print("y_test shape", y_test.shape)


#  Leakage example 1:
### Train on whole data:

In [ ]:
# Train our model on the whole data (Fig. 2 in Sasse et al., 2025)
dt_raw_lkg1 = RandomForestClassifier(max_depth=10, random_state=random_state)
dt_raw_lkg1.fit(X, y)

print("Raw Data - Train accuracy:", dt_raw_lkg1.score(X, y))
print("Raw Data - Test accuracy:", dt_raw_lkg1.score(X_test, y_test))

### Correct procedure:

In [ ]:
# Train our model on the train set and test on the test set
dt_raw = RandomForestClassifier(max_depth=10, random_state=random_state)
dt_raw.fit(X_train, y_train)

print("Raw Data - Train accuracy:", dt_raw.score(X_train, y_train))
print("Raw Data - Test accuracy:", dt_raw.score(X_test, y_test))

When the model was trained in train set and tested on test set the test performance dropped.
When the model was trained in the whole dataset it performed well in both, train and test datasets. 
This is because the model learned patterns of the test set during training.

# Leakage example 2:
### Feature selection on whole dataset:

In [ ]:

# Define reproducible cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

# Feature selection on entire dataset (leakage)
n_components = 7
selector = PCA(n_components=n_components)

X_selected_leak = selector.fit_transform(X.copy(), y)

# Scale and reduce dimensionality on entire dataset (leakage)
scaler = StandardScaler()
X_scaled_leak = scaler.fit_transform(X_selected_leak)

# Evaluate using fixed CV
model = DecisionTreeClassifier(max_depth=10, random_state=random_state)
scores_leakage = []
for train, test in cv.split(X, y):
    model.fit(X_scaled_leak[train, :], y[train])
    pred = model.predict(X_scaled_leak[test, :])
    scores_leakage.append(roc_auc_score(y[test], pred))

### Correct procedure:

In [ ]:
# Pipeline with feature selection inside each fold
selector = PCA(n_components=n_components)
scaler = StandardScaler()
model = DecisionTreeClassifier(max_depth=10, random_state=random_state)

scores_no_leakage = []

for train, test in cv.split(X, y):
    X_train = X[train, :]
    y_train = y[train]
    X_test = X[test, :]
    y_test = y[test]
    
    # Scale and reduce dimensionality on entire dataset (leakage)
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Fit feature selector
    X_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)

    # Fit ML model
    model.fit(X_selected, y_train)

    pred = model.predict(X_test_selected)
    scores_no_leakage.append(roc_auc_score(y[test], pred))

In [ ]:
# ========== Compare Results ==========
results_df = pd.DataFrame(
    {
        "Fold": np.arange(1, 6),
        "Accuracy with Leakage": scores_leakage,
        "Accuracy without Leakage": scores_no_leakage,
        "Difference": np.array(scores_leakage) - np.array(scores_no_leakage),
    }
)

print(results_df)
print("\nMean Accuracy with Leakage: ", round(np.mean(scores_leakage)*100, 4))
print("Mean Accuracy without Leakage: ", round(np.mean(scores_no_leakage)*100, 4))
print("Mean Difference: ", round(np.mean(results_df["Difference"]), 4))

The approach causing leakage generally yielded better performance than the correct approach. Even though in this example the effect of leakage is not huge, in bigger and complex datasets its effect is much severe.

It is important to note that the results and the effect of leakage might change based on the use of different models, seeds, samples, features, etc.
Also leakage is complex and it is often unclear where it might or might not show. However, it is always important to avoid it in order to yield valid estimations of model performance.

# To do!
Lets try another model, another pre-processing, or even random seeds to see the effect of the data leakage.